In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../../bigdata/MIMIC/NOTEEVENTS.csv')

/home/jainsarthak/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
import re

In [21]:
p = re.compile(r'(\[\*\*[^*]*\*\*\])')

In [22]:
texts = list(df['TEXT'])

In [23]:
from tqdm import tqdm_notebook

In [24]:
wardname = (re.compile(r'wardname', re.IGNORECASE), 'Ward')

doctor =  (re.compile(r'doctor', re.IGNORECASE), 'Doctor')
firstname = (re.compile(r'firstname|(First Name)', re.IGNORECASE), "First")
lastname = (re.compile(r'lastname|(Last Name)', re.IGNORECASE), "Last")
patient = (re.compile(r'patient', re.IGNORECASE), "Patient")
stitle = (re.compile(r'\(\s*STitle\s*\)', re.IGNORECASE), "STitle")
ptitle = (re.compile(r'\(\s*PTitle\s*\)', re.IGNORECASE), "PTitle")
prefixes = (re.compile(r'\(\s*Prefixes\s*\)', re.IGNORECASE), "Prefixes")
md = (re.compile(r'\(\s*MD\s*\)', re.IGNORECASE), "MD")
pre = (re.compile(r'\(\s*PRE\s*\)', re.IGNORECASE), "PRE")

name_patterns = [wardname, doctor, patient, firstname, lastname, stitle, ptitle, prefixes, md, pre]
def process_name(name) :
    s = ''
    for reg, sub in name_patterns :
        if reg.search(name) is not None :
            s += 'MIMIC_' + sub + '_MIMIC '
        
    s += 'MIMIC_Name_MIMIC'
    return s

In [25]:
name = (re.compile(r'name', re.IGNORECASE), 'Name')
date = (re.compile(r'\[\*\*(\d+-\d+-\d+)|(.*month.*)|(.*year.*)|(.*day.*)|(.*date.*)\*\*\]', re.IGNORECASE), 'Date')
clip = (re.compile(r'clip', re.IGNORECASE), 'Clip_Number')
jobnum = (re.compile(r'job', re.IGNORECASE), 'Job_Number')
telephone = (re.compile(r'telephone|fax', re.IGNORECASE), 'Telephone')
location = (re.compile(r'location', re.IGNORECASE), 'Location')
hospital = (re.compile(r'hospital', re.IGNORECASE), 'Hospital')
numid = (re.compile(r'numeric', re.IGNORECASE), 'NumID')
pager = (re.compile(r'pager', re.IGNORECASE), 'Pager')
mdnumber = (re.compile(r'md number', re.IGNORECASE), 'MDNumber')
medrec = (re.compile(r'medical record number', re.IGNORECASE), 'MedRecNum')
email = (re.compile(r'e-mail', re.IGNORECASE), 'E-Mail')
sernum = (re.compile(r'serial number', re.IGNORECASE), 'SerialNum')
country = (re.compile(r'country', re.IGNORECASE), 'Country')
state = (re.compile(r'state', re.IGNORECASE), 'State')
street = (re.compile(r'street|apartment', re.IGNORECASE), 'Addr')
ssn = (re.compile(r'social security', re.IGNORECASE), 'SSN')
univ = (re.compile(r'university', re.IGNORECASE), 'Univ')
company = (re.compile(r'company', re.IGNORECASE), 'Company')
url = (re.compile(r'url', re.IGNORECASE), 'URL')
ageover = (re.compile(r'age over', re.IGNORECASE), 'AgeOver90')
ccontact = (re.compile(r'cc contact', re.IGNORECASE), 'CCContact')
dictinfo = (re.compile(r'dictator', re.IGNORECASE), 'Dictator')
unitnum = (re.compile(r'unit number', re.IGNORECASE), 'Unit')
provider = (re.compile(r'provider', re.IGNORECASE), 'Provider')
datebad = (re.compile(r'(January|February|March|April|May|June|July|August|September|October|November|December)', 
                      re.IGNORECASE), 'Date')

measurement = (re.compile(r'(\d+-\d+)|(-?\d+-?/\d+)|(\d+-)', re.IGNORECASE), 'Measurement')
pobox = (re.compile(r'po box', re.IGNORECASE), 'POBox')
attending = (re.compile(r'attending', re.IGNORECASE), 'Attending')
num = (re.compile(r'\[\*\*\s*(\d+)\s*\*\*\]', re.IGNORECASE), 'Numeric')
ethnicity = (re.compile(r'ethnicity', re.IGNORECASE), 'Ethnicity')
empty = (re.compile(r'\[\*\*\s*\*\*\]', re.IGNORECASE), 'Empty')

regexs = [date, clip, 
          jobnum, telephone, location, hospital, 
          numid, pager, mdnumber, medrec, 
          email, sernum, country, street, ssn, state,
          univ, company, url, ageover, ccontact, dictinfo, 
          datebad, unitnum, provider, pobox, attending, ethnicity, measurement, num, empty]

In [26]:
def process_deid(deid) :
    if name[0].search(deid) is not None :
        deid = process_name(deid)
    else :
        for reg, sub in regexs :
            if reg.search(deid) is not None :
                deid = sub
                break
        
        deid = 'MIMIC_' + deid + '_MIMIC'
    return deid

In [27]:
deid = []
for t in tqdm_notebook(texts) :
    deid.append(p.sub(lambda m : process_deid(m.group(0)), t))

In [28]:
for i in tqdm_notebook(range(len(deid))) :
    deid[i] = re.sub(r'\n+', '\n', deid[i])

In [29]:
for i in tqdm_notebook(range(len(deid))) :
    deid[i] = re.sub(r'[ \t]+', ' ', deid[i])

In [30]:
total_deid = []
q = re.compile(r'(MIMIC_.*?_MIMIC)')
for t in deid :
    total_deid += q.findall(t)

In [41]:
total_deid = pd.Series(pd.Series(total_deid).unique())

In [42]:
pd.Series(total_deid.unique()).to_csv("Deidentifiers_map.csv", index=False)

In [69]:
for i in tqdm_notebook(range(len(deid))) :
    f = open('jaydata/Note_' + str(i) + '.txt', 'w')
    f.write(deid[i])
    f.close()

In [47]:
import tarfile
from io import BytesIO
data = "this is a UTF-8 string"
with tarfile.open("jaydata.tar.gz", 'w:gz') as of:
  # turn data into bytes, this might be the wrong way; decode might be more appropriate
    for i in tqdm_notebook(range(len(deid))) :
        writeable_data = deid[i].encode()
        tarinfo = tarfile.TarInfo(name='jaydata/Note_' + str(i) + '.txt')
        tarinfo.size = len(writeable_data)
        of.addfile(tarinfo, BytesIO(writeable_data))